In [4]:
import urllib.request
import os
import pandas as pd
from datetime import datetime
import glob
import sys

In [5]:

DATA_DIR = "noaa_data"
os.makedirs(DATA_DIR, exist_ok=True)

def download_vhi(province_id):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"

    try:
        wp = urllib.request.urlopen(url)
        text = wp.read().decode("utf-8")
        from io import StringIO
        headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
        new_data = pd.read_csv(StringIO(text), header = 1,  names = headers, sep=",")

        existing_files = glob.glob(os.path.join(DATA_DIR, f"obl_{province_id}_*.csv"))
        if existing_files:
            existing_file = existing_files[0]  
            old_data = pd.read_csv(existing_file, header = 1, names = headers)

            if new_data.equals(old_data):
                print(f" Дані для області {province_id} не змінилися. Файл залишився без змін.")
                return

            os.remove(existing_file)

        current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")
        new_file_path = os.path.join(DATA_DIR, f"obl_{province_id}_{current_datetime}.csv")

        new_data.to_csv(new_file_path, index=False)
        print(f" Дані оновлені. Новий файл: {new_file_path}")

    except Exception as e:
        print(f" Помилка при завантаженні області {province_id}: {e}")



for i in range(1, 29):
    province_id = i
    download_vhi(province_id)



 Дані оновлені. Новий файл: noaa_data\obl_1_15032025131332.csv
 Дані оновлені. Новий файл: noaa_data\obl_2_15032025131333.csv
 Дані оновлені. Новий файл: noaa_data\obl_3_15032025131337.csv
 Дані оновлені. Новий файл: noaa_data\obl_4_15032025131339.csv
 Дані оновлені. Новий файл: noaa_data\obl_5_15032025131341.csv
 Дані оновлені. Новий файл: noaa_data\obl_6_15032025131342.csv
 Дані оновлені. Новий файл: noaa_data\obl_7_15032025131343.csv
 Дані оновлені. Новий файл: noaa_data\obl_8_15032025131344.csv
 Дані оновлені. Новий файл: noaa_data\obl_9_15032025131345.csv
 Дані оновлені. Новий файл: noaa_data\obl_10_15032025131348.csv
 Дані оновлені. Новий файл: noaa_data\obl_11_15032025131349.csv
 Дані оновлені. Новий файл: noaa_data\obl_12_15032025131350.csv
 Дані оновлені. Новий файл: noaa_data\obl_13_15032025131351.csv
 Дані оновлені. Новий файл: noaa_data\obl_14_15032025131352.csv
 Дані оновлені. Новий файл: noaa_data\obl_15_15032025131353.csv
 Дані оновлені. Новий файл: noaa_data\obl_16_1503

In [8]:

def load_vhi_data(directory):
    all_files = [f for f in os.listdir(directory) if f.startswith("obl_") and f.endswith(".csv")]
    
    if not all_files:
        print("Немає файлів для завантаження!")
        return None
    
    dataframes = []
    
    for file in all_files:
        file_path = os.path.join(directory, file)
        headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
        df = pd.read_csv(file_path, header = 1, names = headers)
        
        
        province_id = int(file.split("_")[1])
        df.insert(0, "Province_ID", province_id)
        
        dataframes.append(df)
    
    final_df = pd.concat(dataframes, ignore_index=True)
    print("Завантажено", len(all_files), "файлів.")
    return final_df

DATA_DIR = "noaa_data"
df_vhi = load_vhi_data(DATA_DIR)
if df_vhi is not None:
    print(df_vhi.head())


Завантажено 28 файлів.
   Province_ID  Year  Week    SMN     SMT    VCI    TCI    VHI  empty
0           10  1982   2.0  0.063  261.53  55.89  38.20  47.04    NaN
1           10  1982   3.0  0.063  263.45  57.30  32.69  44.99    NaN
2           10  1982   4.0  0.061  265.10  53.96  28.62  41.29    NaN
3           10  1982   5.0  0.058  266.42  46.87  28.57  37.72    NaN
4           10  1982   6.0  0.056  267.47  39.55  30.27  34.91    NaN


In [6]:
def rename_province_ids(df):
    province_mapping = {
        1: 13, 2: 14, 3: 15, 4: 16, 5: 17, 6: 18, 7: 19, 8: 20,
        9: 21, 10: 22, 11: 23, 12: 24, 13: 1, 14: 2, 15: 3, 16: 4,
        17: 5, 18: 6, 19: 7, 20: 8, 21: 9, 22: 10, 23: 11, 24: 12,
        25: 25  
    }
    
    df["Province_ID"] = df["Province_ID"].replace(province_mapping)
    
    print("Готово!\n", df.head())  
    return df

df_vhi = rename_province_ids(df_vhi)



NameError: name 'df_vhi' is not defined

In [1]:
def get_vhi_by_year(df, province_id, year):
    return df[(df["Province_ID"] == province_id) & (df["Year"] == year)]
def get_extremes(df, province_ids, years):
    df_vhi["Year"] = pd.to_numeric(df_vhi["Year"], errors="coerce").astype("Int64")
    subset = df[(df["Province_ID"].isin(province_ids)) & (df["Year"].isin(years))]
    return {
        "min": subset["VHI"].min(),
        "max": subset["VHI"].max(),
        "mean": subset["VHI"].mean(),
        "median": subset["VHI"].median()
    }
def get_vhi_by_year_range(df, province_ids, start_year, end_year):
    return df[(df["Province_ID"].isin(province_ids)) & (df["Year"].between(start_year, end_year))]

def find_drought_years(df, threshold=5):
    drought_years = []
    for year in df["Year"].unique():
        yearly_data = df[df["Year"] == year]
        drought_regions = yearly_data[yearly_data["VHI"] < 15]["Province_ID"].unique()
        if len(drought_regions) >= threshold:
            drought_years.append((year, drought_regions))
    return drought_years